# Model Forecasting

## Requirements

In [9]:
#import necesary libraries
import pandas as pd
import os
import pickle
import gc
from matplotlib.backends.backend_pdf import PdfPages

import matplotlib.pyplot as plt 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import plotly.io as pio

from utils import load

## Data Load

In [10]:
df, resample_df = load()

## ARIMA Update & Forecasting

In [19]:
#for cycle to iterate over folder
    #if ends with .pkl
        #load model
        #update model
        #forecast
        #plot

#accesing .pkl files
route = '../stock_forecasting/AutoARIMA_Models/PKL/'
directory = os.fsencode(route)
pdf = PdfPages('../stock_forecasting/AutoARIMA_Models/Forecast/viz.pdf')

window = 8 #number of days to forecast
cont = 0

for file in os.listdir(directory): #for cycle to iterate over folder
   filename = os.fsdecode(file) #get file name
   if filename.endswith(".pkl"): #if ends with .pkl
      cont+=1
      print('Forecasting for:',filename,'(',cont,'/ 31 )') #FLAG
      
      with open((route+filename).replace(" ",""), 'rb') as f: #load model
         model = pickle.load(f)

      x = filename.split("_") #use this snippet to solve df slicer function... look for NB_EDA...
      print(x[0])
      print('')

      #model.update()# LOAD DF WITH NEW DATES 

      '''
      fig, ax = plt.subplots(figsize=(15, 8))
      ax.set_title(hotel + ' - ' + target)
      df[(df['vn_name']==hotel)][target].asfreq('D').tail(last_window).plot() 
      data.predict(steps = last_window, last_window = df[(df['vn_name']==hotel)][target].asfreq('D').tail(last_window)).plot(ax=ax)
      pdf.savefig(fig, bbox_inches='tight')
      plt.close(fig) #optimize ram usage
pdf.close()
'''

Forecasting for: MMM_autoARIMA_Model.pkl ( 1 / 31 )
MMM

Forecasting for: AABA_autoARIMA_Model.pkl ( 2 / 31 )
AABA

Forecasting for: DIS_autoARIMA_Model.pkl ( 3 / 31 )
DIS

Forecasting for: UTX_autoARIMA_Model.pkl ( 4 / 31 )
UTX

Forecasting for: XOM_autoARIMA_Model.pkl ( 5 / 31 )
XOM

Forecasting for: AMZN_autoARIMA_Model.pkl ( 6 / 31 )
AMZN

Forecasting for: TRV_autoARIMA_Model.pkl ( 7 / 31 )
TRV

Forecasting for: GS_autoARIMA_Model.pkl ( 8 / 31 )
GS

Forecasting for: AXP_autoARIMA_Model.pkl ( 9 / 31 )
AXP

Forecasting for: IBM_autoARIMA_Model.pkl ( 10 / 31 )
IBM

Forecasting for: UNH_autoARIMA_Model.pkl ( 11 / 31 )
UNH

Forecasting for: WMT_autoARIMA_Model.pkl ( 12 / 31 )
WMT

Forecasting for: PFE_autoARIMA_Model.pkl ( 13 / 31 )
PFE

Forecasting for: NKE_autoARIMA_Model.pkl ( 14 / 31 )
NKE

Forecasting for: CSCO_autoARIMA_Model.pkl ( 15 / 31 )
CSCO

Forecasting for: CAT_autoARIMA_Model.pkl ( 16 / 31 )
CAT

Forecasting for: INTC_autoARIMA_Model.pkl ( 17 / 31 )
INTC

Forecasting for: 